<a href="https://colab.research.google.com/github/ShakutaiGit/Final_Project_Applied_Data_science/blob/main/Souce_Code_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

relevant imports 

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers.experimental import preprocessing
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model 
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization
from tensorflow.python.keras.layers.convolutional import Conv2D, MaxPooling2D 
import numpy as np
import pandas as pd 


global variables

In [2]:
optimizer = None
size = 224 
batch_size = 32 # like the original paper 
classes = 102
AUTOTUNE = tf.data.AUTOTUNE

loading the dataset

In [3]:

train_ds ,train_info= tfds.load('oxford_flowers102', split='train[:80%]', with_info=True,batch_size=128)
val_ds,val_info = tfds.load('oxford_flowers102', split='train[80%:90%]', with_info=True,batch_size=128)
test_ds , test_info = tfds.load('oxford_flowers102', split='train[90%:]', with_info=True)



Shuffling and writing examples to /root/tensorflow_datasets/oxford_flowers102/2.1.1.incompleteIDQP1H/oxford_flowers102-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/oxford_flowers102/2.1.1.incompleteIDQP1H/oxford_flowers102-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/oxford_flowers102/2.1.1.incompleteIDQP1H/oxford_flowers102-validation.tfrecord


Dataset oxford_flowers102 downloaded and prepared to /root/tensorflow_datasets/oxford_flowers102/2.1.1. Subsequent calls will reuse this data.


preprocessing

In [5]:
# pre processing the data 

# augmentation function 
data_augmentation = tf.keras.Sequential([

            layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
            layers.experimental.preprocessing.RandomRotation(0.2),
            layers.experimental.preprocessing.RandomZoom(0.1)
])
# normlizing and resizing function 
resize_and_rescale = tf.keras.Sequential([
layers.experimental.preprocessing.Resizing(size, size)
])

def prepare(ds):
  # Resize and rescale all kind datasets
  ds = ds.map(lambda data: (data['image'], tf.one_hot(data['label'], 102)), 
              num_parallel_calls=AUTOTUNE)
  # Batch all datasets

  return ds

creating the optimizer and define the warm restarts variables

In [6]:
first_decay_steps = 5
initial_learning_rate=0.1
lr_decayed_fn = tf.keras.optimizers.schedules.CosineDecayRestarts(
initial_learning_rate, first_decay_steps, t_mul=2.0, m_mul=1.0, alpha=0.0)
optimizer = tf.keras.optimizers.SGD(learning_rate=lr_decayed_fn)

building the model 

In [7]:
def creating_the_model():
  inputs =  layers.Input(shape=(224,224,3))
  x=resize_and_rescale(inputs)
  x=data_augmentation(x)
  pre_trained_model = keras.applications.ResNet50(include_top=False, weights='imagenet',input_tensor=x,pooling='avg',classes=classes)
  pre_trained_model.trainable=False
  x = layers.BatchNormalization()(pre_trained_model.output)
  top_dropout_rate = 0.5
  x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
  outputs = layers.Dense(102, activation="softmax", name="pred")(x)
  model = tf.keras.Model(inputs, outputs, name="ResNet")

  #un freeze all the layers with out batch normlization 
  for layer in model.layers[-8:]:
    if not isinstance(layer, layers.BatchNormalization) :
        layer.trainable = True
  return model


In [8]:
model = creating_the_model()

94773248/94765736 [==============================] - 1s 0us/step


In [9]:
def compile_fit_and_stuff(train_ds,val_ds):
  train_ds=prepare(train_ds)
  val_ds=prepare(val_ds)
  model.summary()
  model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["acc"])
  model.fit(train_ds,epochs=3,validation_data=val_ds)

hyper parameter & train the model 

In [10]:
compile_fit_and_stuff(train_ds,val_ds)

Model: "ResNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
sequential_3 (Sequential)       (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
sequential_2 (Sequential)       (None, 224, 224, 3)  0           sequential_3[0][0]               
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           sequential_2[0][0]               
_____________________________________________________________________________________________

results evaluation

In [4]:
targets = [0,1,2,0,1,2,0,1,2]

outputs = [0,1,2,0,0,2,1,1,1]
print(outputs[-3:])

[1, 1, 1]


saving the results 